After selection bicone, in this notebook we plot the OIII maps to verify if our set is good. 

1. Verify the candidate list
2. Verify the bicone index

In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from marvin.tools import Maps

from tqdm import tqdm



[INFO]: No release version set. Setting default to DR17
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /Users/runquanguan/sas/mangawork/manga/spectro/analysis/v3_1_1/3.1.0/dapall-v3_1_1-3.1.0.fits cannot be found. Setting dapall to None. (MarvinUserWarning)
[WARNING]: Passing field metadata as keyword arguments is deprecated. Use the explicit `metadata=...` argument instead. Additional metadata: {'location': 'view_args'} (RemovedInMarshmallow4Warning)
[WARNING]: The 'missing' argument to fields is deprecated. Use 'load_default' instead. (RemovedInMarshmallow4Warning)


In [4]:
HEADER_PATH = '/Users/runquanguan/Documents/new_and_full.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['plateifu']


for data in tqdm(plateifu):
    # Load OIII MAPS from SDSS-MARVIN server
    maps = Maps(data, bintype='SPX', template='MILESHC-MASTARSSP')
    oiii_ew = maps.emline_gew_oiii_5008
    oiii_ew.plot()
    plt.savefig(data+'.png')




[WARNING]: unclosed file <_io.FileIO name='/Users/runquanguan/Documents/new_and_full.fits' mode='rb' closefd=True> (ResourceWarning)
100%|██████████| 246/246 [08:06<00:00,  1.98s/it]


In [12]:
PATH = '/Users/runquanguan/Documents/Research/MaNGA-AGN/Pipeline&Instrction/obvious_bicone_feature_position.fits'
hdul = fits.open(PATH)
hdu = hdul[1].data['plateifu']

for data in hdu:
    # Load OIII MAPS from SDSS-MARVIN server
    maps = Maps(data, bintype='SPX', template='MILESHC-MASTARSSP')
    oiii_ew = maps.emline_gew_oiii_5008
    oiii_ew.plot()
    plt.savefig(data+'.png')


[WARNING]: unclosed file <_io.FileIO name='/Users/runquanguan/Documents/bicone_candidates_v5.fits' mode='rb' closefd=True> (ResourceWarning)


## Now clean the fits file a bit

 turn index into probability, 
 
 turn loss into degree_sig
 

In [14]:
PATH = '/Users/runquanguan/Documents/Research/MaNGA-AGN/bicone_candidates_Full_v1.fits'
hdul = fits.open(PATH)
hdu = hdul[1].data['plateifu']
index = hdul[1].data['BiconeIndex']

In [20]:
float(max(index))

829.6152806599077

In [22]:
bicone_proba = []
for i in index:
    proba = float(i)/float(max(index))
    bicone_proba.append(proba)

In [23]:
bicone_proba

[0.6435830483851926,
 0.27531894832344217,
 0.918000891228216,
 0.3541095366585754,
 0.5255146644066642,
 0.5003756777160263,
 0.557084337908012,
 0.279317805940235,
 0.4029764304065121,
 0.6877628755066905,
 0.3083580234552602,
 0.29878580578221947,
 0.2762707561015234,
 0.2742766999630022,
 0.43513876451506045,
 0.596195843647316,
 0.2698667364689344,
 0.7382467540966681,
 0.27881241016170344,
 0.3140637016937561,
 0.6050425834378053,
 0.5528286438691697,
 0.3510642914495778,
 0.7375005614380377,
 0.3568697716051151,
 0.38970246046150075,
 0.8824225674739464,
 0.32669669634972787,
 0.36893771897751,
 0.2824669057217317,
 0.3295974603456429,
 0.3460011687165712,
 0.28006237792185007,
 0.5929670949066496,
 0.5016694707342988,
 0.4874723007236675,
 0.6466671228420854,
 0.319466765096255,
 0.3041110098282087,
 0.5500171158657817,
 0.39666122031635814,
 0.43269667763920633,
 0.46160130386219944,
 0.4697645385524186,
 0.4106335266265782,
 0.37148852312947106,
 0.5073549160754639,
 0.331744

In [24]:
c1 = fits.Column(name='plateifu', array=hdu, format='20A')
c2 = fits.Column(name='BiconeProbability', array=np.array(bicone_proba), format='E')
t = fits.BinTableHDU.from_columns([c1, c2])
t.writeto('bicone_candidates_Full_v3.fits')

In [30]:
PATH = '/Users/runquanguan/Documents/bicone_candidates_Full_v2.fits'
hdul = fits.open(PATH)
hdu = hdul[1].data['plateifu']
index = hdul[1].data['BiconeIndex']

In [31]:
degree_sig = []
for i in index:
    sig = float(i)/8
    degree_sig.append(sig)

In [32]:
degree_sig

[12.960000000000006,
 22.320000000000004,
 11.52000000000001,
 17.640000000000008,
 20.52000000000003,
 14.940000000000012,
 27.900000000000016,
 23.580000000000005,
 7.200000000000024,
 20.340000000000007,
 12.600000000000016,
 6.84,
 6.300000000000011,
 26.099999999999998,
 26.640000000000022,
 27.54000000000001,
 11.880000000000017,
 16.920000000000016,
 6.660000000000011,
 17.10000000000001,
 18.540000000000006,
 14.580000000000023,
 24.300000000000008,
 24.66000000000001,
 19.440000000000005,
 22.320000000000007,
 18.539999999999996,
 9.54000000000001,
 16.02,
 26.280000000000012,
 26.46000000000002,
 19.980000000000018,
 25.02,
 18.900000000000006,
 19.62000000000002,
 16.74000000000001,
 10.979999999999997,
 13.320000000000022,
 17.280000000000022,
 17.64,
 8.820000000000007,
 27.720000000000002,
 4.140000000000008,
 19.260000000000012,
 24.12000000000001,
 7.5600000000000165,
 13.680000000000012,
 11.159999999999993,
 19.440000000000005,
 17.460000000000004,
 16.020000000000017

In [34]:
c1 = fits.Column(name='plateifu', array=hdu, format='20A')
c2 = fits.Column(name='degree_sig', array=np.array(degree_sig), format='E')
t = fits.BinTableHDU.from_columns([c1, c2])
t.writeto('bicone_candidates_Full_v4.fits')

In [35]:
HEADER_PATH = '/Users/runquanguan/Documents/Research/MaNGA-AGN/test_v1.fits'
hdul = fits.open(HEADER_PATH)
hdu = hdul[1].data
plateifu = hdu['PLATEIFU']

#fig, axes = plt.subplots (nrows=4, ncols=5,figsize =(12,11))

for i in plateifu:
    maps = Maps(i, bintype='SPX', template='MILESHC-MASTARSSP')
    oiii_ew = maps.emline_gew_oiii_5008
    oiii_ew.plot(title = i, snr_min = None)
    plt.savefig(i+'.png')

[WARNING]: unclosed file <_io.FileIO name='/Users/runquanguan/Documents/bicone_candidates_Full_v2.fits' mode='rb' closefd=True> (ResourceWarning)
[WARNING]: unclosed file <_io.FileIO name='/Users/runquanguan/Documents/Research/MaNGA-AGN/bicone_candidates_Full_v1.fits' mode='rb' closefd=True> (ResourceWarning)
[WARNING]: unclosed file <_io.FileIO name='/Users/runquanguan/Documents/Research/MaNGA-AGN/bicone_candidates_Full_v1.fits' mode='rb' closefd=True> (ResourceWarning)
[WARNING]: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). (RuntimeWarning)
